We use the Protogen flood water mapper on a collection of images in Colorado, Louisiana, North Carolina and Madagascar where flooding has occurred in the past. The following GBDX workflow extracts the flooded regions from atmospherically compensated 8-band imagery and generates the corresponding Mapbox raster tilesets for viewing purposes. 

Define environment variables and create gbdx interface.

In [ ]:
import gbdxtools
import os

# gbdx credentials
os.environ['GBDX_USERNAME'] = ''
os.environ['GBDX_PASSWORD'] = ''
os.environ['GBDX_CLIENT_ID'] = '' 
os.environ['GBDX_CLIENT_SECRET'] = ''

# mapbox secret token
mapbox_token = ''

gbdx = gbdxtools.Interface()

Specify material for this workflow.

In [ ]:
# overall location
s3_location = 's3://gbd-customer-data/58600248-2927-4523-b44b-5fec3d278c09/platform-stories/flood-water/'

# catalog ids per place
cat_idss = {'baton-rouge': ['103001005B078B00', '103001005B07E700', '103001005B38CE00', '103001005B3EEE00', '103001005C86EC00', '103001005D662200'], 
            'foothills': ['103001002608E100', '1030010026102200', '1030010026CFCF00', '1030010027317F00', '1030010027BFBF00'], 
            'north-carolina': ['103001005D499E00', '103001005E9A0300', '10400100238BDE00'], 
            'maroantsetra': ['1050010008989C00']}



Create protogen-based flood water mapping task.

In [ ]:
import protogen
import pickle

p = protogen.Interface('crime', 'flood_water_mapper')
p.crime.flood_water_mapper.tolerance = 50               # [%] aim for water bodies only
p.crime.flood_water_mapper.moisture = 3                 # [%] retain all candidates
p.crime.flood_water_mapper.binary_output = True         # output format
p.crime.flood_water_mapper.min_size = 1000              # noise: average housing roof size in square meters 
p.crime.flood_water_mapper.min_width = 10               # noise: road width in meters
p.image_config.bands = [1, 2, 3, 4, 5, 6, 7, 8]
p.verbose = True

fld = gbdx.Task('protogen-runner')
fld.inputs.pickle = pickle.dumps(p)

Define and run the workflow. Note that no data is saved on s3! The flood map and the corresponding pansharpened image are uploaded to mapbox with [upload-to-mapbox](https://github.com/PlatformStories/upload-to-mapbox) so that the tilesets used in the slippy map are created. Note the use of [modify-colors](https://github.com/PlatformStories/modify-colors) to manipulate the colors of the black-and-white masks generated by the flood mapper. 

In [ ]:
from os.path import join

wf_ids = {}

for place in cat_idss.keys():
    
    for i, cat_id in enumerate(cat_idss[place]):
  
        # locations of acomped multispectral image and corresponding pansharpened image
        ms_location = join(s3_location, place, cat_id, 'ms')
        ps_location = join(s3_location, place, cat_id, 'ps')
        
        # specify input to flood mapper
        fld.inputs.image = ms_location
        
        # the output of the flood mapper is a binary image where white = flood and black = background
        # use the modify-colors task to change white to brown and black to transparent
        mc = gbdx.Task('modify-colors')
        mc.inputs.image = fld.outputs.output.value
        mc.inputs.transparency = 'True'
        mc.inputs.colors = '255,255,255:165,42,42'

        # compress pansharpened image for visualization purposes
        compress = gbdx.Task('gdal-cli')
        compress.inputs.data = ps_location
        compress.inputs.execution_strategy = 'runonce'
        compress.inputs.command = 'gdal_translate  $indir/*/*.tif $outdir/compressed.tif -b 1 -b 2 -b 3 -co COMPRESS=JPEG -co PHOTOMETRIC=YCBCR -co TILED=YES'

        # upload to mapbox
        tileset_name_prefix = '-'.join(['flood-water', place, str(i)])
        # mask
        utom_mask = gbdx.Task('upload-to-mapbox')
        utom_mask.inputs.input = mc.outputs.output.value
        utom_mask.inputs.tileset_name = '-'.join([tileset_name_prefix, 'm'])
        utom_mask.inputs.token = mapbox_token
        # pansharpened
        utom_ps = gbdx.Task('upload-to-mapbox')
        utom_ps.inputs.input = compress.outputs.data.value
        utom_ps.inputs.tileset_name = tileset_name_prefix 
        utom_ps.inputs.token = mapbox_token
        utom_ps.inputs.timeout = '1200'       # increase timeout for pansharpened image in case of big files

        # create two-task preprocessing workflow
        #wf = gbdx.Workflow([fld, mc, compress, utom_mask, utom_ps])
        wf = gbdx.Workflow([fld, mc, utom_mask])
        
        wf_ids[cat_id] = wf.execute()


In [ ]:
# check status of workflows    
for cat_id in wf_ids.keys():
    wf = gbdx.Workflow([])
    wf.id = wf_ids[cat_id]
    print wf.id, wf.status